In [9]:
import pyodbc
import pandas as pd

conn = pyodbc.connect(
    "DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;"
    "SERVER=localhost,1433;"
    "DATABASE=CMU_DS_WP;"
    "UID=sa;"
    "PWD=YourStrong!Passw0rd;"
    "Encrypt=no;"
    "TrustServerCertificate=yes;"
)

df = pd.read_sql("SELECT TOP 5 * FROM sys.tables", conn)
print(df.head())

/var/folders/hs/r4ck14j54v17d80mrc1wt5kw0000gn/T/ipykernel_87651/3966665983.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT TOP 5 * FROM sys.tables", conn)


           name   object_id principal_id  schema_id  parent_object_id type  \
0   WP_Products   997578592         None          1                 0   U    
1   WP_Activity  1013578649         None          1                 0   U    
2  WP_Locations  1029578706         None          1                 0   U    
3   RN_Activity  1061578820         None          1                 0   U    
4  RN_Locations  1077578877         None          1                 0   U    

    type_desc             create_date             modify_date  is_ms_shipped  \
0  USER_TABLE 2025-12-22 09:50:45.880 2025-12-22 09:50:45.893          False   
1  USER_TABLE 2025-12-22 09:51:45.253 2025-12-22 09:57:11.507          False   
2  USER_TABLE 2025-12-22 09:54:07.023 2025-12-22 09:54:07.033          False   
3  USER_TABLE 2025-12-22 10:03:14.497 2025-12-22 10:03:14.497          False   
4  USER_TABLE 2025-12-22 10:03:33.957 2025-12-22 10:03:33.957          False   

   ...  history_retention_period_unit_desc  is_nod

In [ ]:
df = pd.read_sql(
    "SELECT TOP 10 * FROM dbo.OE_Activity",
    conn
)
df

/var/folders/hs/r4ck14j54v17d80mrc1wt5kw0000gn/T/ipykernel_87651/4005373291.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


,ActivityCode,UserID,WorkCode,AssignmentID,ProductID,Quantity,Timestamp,LocationID
0,PickPut,419,20,7954566,4289.0,1.0,2025-11-10 11:37:14.160,826367.0
1,AssignmentOpen,64,10,7954429,NaN,NaN,2025-11-10 11:38:34.043,NaN
2,PickPut,419,20,7954541,6592.0,1.0,2025-11-10 11:39:42.330,14524.0
3,PickPut,419,20,7954542,6592.0,1.0,2025-11-10 11:39:42.883,14524.0
4,PickPut,419,20,7954543,6592.0,1.0,2025-11-10 11:39:43.483,14524.0
5,PickPut,419,20,7954544,6592.0,1.0,2025-11-10 11:39:43.863,14524.0
6,PickPut,419,20,7954545,6592.0,1.0,2025-11-10 11:39:44.460,14524.0
7,PickPut,419,20,7954546,6592.0,1.0,2025-11-10 11:39:44.823,14524.0
8,PickPut,475,20,7954666,4289.0,1.0,2025-11-10 11:39:57.997,826367.0
9,PickPut,475,20,7954681,4289.0,1.0,2025-11-10 11:39:58.170,826367.0


In [ ]:
row_count = pd.read_sql(
    "SELECT COUNT(*) AS total_rows FROM dbo.OE_Activity",
    conn
)

row_count

/var/folders/hs/r4ck14j54v17d80mrc1wt5kw0000gn/T/ipykernel_87651/2119998872.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  row_count = pd.read_sql(


,total_rows
0,96132


In [ ]:
query = """
SELECT
    t.name AS table_name,
    SUM(p.rows) AS num_rows,
    COUNT(c.column_id) AS num_columns
FROM sys.tables t
JOIN sys.schemas s
    ON t.schema_id = s.schema_id
JOIN sys.partitions p
    ON t.object_id = p.object_id
JOIN sys.columns c
    ON t.object_id = c.object_id
WHERE s.name = 'dbo'
  AND p.index_id IN (0,1)   -- heap or clustered index
GROUP BY t.name
ORDER BY t.name;
"""

df_tables = pd.read_sql(query, conn)
df_tables


/var/folders/hs/r4ck14j54v17d80mrc1wt5kw0000gn/T/ipykernel_87651/709917731.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tables = pd.read_sql(query, conn)


,table_name,num_rows,num_columns
0,OE_Activity,769056,8
1,OE_Locations,167595,5
2,OE_Products,461368,8
3,OF_Activity,1019256,8
4,OF_Locations,223855,5
5,OF_Products,522472,8
6,RN_Activity,279928,8
7,RN_Locations,97465,5
8,RN_Products,323184,8
9,RT_Activity,1802864,8


In [ ]:
# Display column names of OE_Activity
query_cols = """
SELECT COLUMN_NAME
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'OE_Activity'
ORDER BY ORDINAL_POSITION
"""
cols_df = pd.read_sql(query_cols, conn)
print(cols_df['COLUMN_NAME'].tolist())

['ActivityCode', 'UserID', 'WorkCode', 'AssignmentID', 'ProductID', 'Quantity', 'Timestamp', 'LocationID']


/var/folders/hs/r4ck14j54v17d80mrc1wt5kw0000gn/T/ipykernel_87651/2085866745.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cols_df = pd.read_sql(query_cols, conn)


In [ ]:
import os

output_dir = "csv"
os.makedirs(output_dir, exist_ok=True)

tables = [
    "OE_Activity",
    "OE_Locations",
    "OE_Products",
    "OF_Activity",
    "OF_Locations",
    "OF_Products",
    "RT_Activity",
    "RT_Locations",
    "RT_Products"
]

CHUNK_SIZE = 100_000

for table in tables:
    print(f"Extracting {table}...")

    query = f"SELECT * FROM {table}"
    output_path = os.path.join(output_dir, f"{table}.csv")

    chunks = pd.read_sql(query, conn, chunksize=CHUNK_SIZE)

    with open(output_path, "w", newline="", encoding="utf-8") as f:
        for i, chunk in enumerate(chunks):
            chunk.to_csv(
                f,
                index=False,
                header=(i == 0)
            )

    print(f"Saved → {output_path}")

print("All tables extracted successfully.")

Extracting OE_Activity...


/var/folders/hs/r4ck14j54v17d80mrc1wt5kw0000gn/T/ipykernel_87651/517329984.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunks = pd.read_sql(query, conn, chunksize=CHUNK_SIZE)


Saved → csv/OE_Activity.csv
Extracting OE_Locations...
Saved → csv/OE_Locations.csv
Extracting OE_Products...
Saved → csv/OE_Products.csv
Extracting OF_Activity...
Saved → csv/OF_Activity.csv
Extracting OF_Locations...
Saved → csv/OF_Locations.csv
Extracting OF_Products...
Saved → csv/OF_Products.csv
Extracting RT_Activity...
Saved → csv/RT_Activity.csv
Extracting RT_Locations...
Saved → csv/RT_Locations.csv
Extracting RT_Products...
Saved → csv/RT_Products.csv
All tables extracted successfully.


In [ ]:
from IPython.display import display

print("Working directory:", os.getcwd())
df = pd.read_csv("distance_matrix_OE.csv")

print("EDA for distance_matrix_OE.csv")

# Dimensions
print(f"Dimensions: {df.shape[0]} rows × {df.shape[1]} columns\n")

# First 5 rows
print("First 5 rows:")
display(df.head())

# Column-level schema summary
schema_df = pd.DataFrame({
    "dtype": df.dtypes.astype(str),
    "n_missing": df.isna().sum(),
    "n_unique": df.nunique(dropna=True)
})
print("\nColumn-level summary:")
display(schema_df)

Working directory: /Users/jiashenwang/Desktop/data
EDA for distance_matrix_OE.csv
Dimensions: 1246 rows × 1247 columns

First 5 rows:


,Unnamed: 0,08|03|||,08|05|||,08|07|||,08|09|||,10|04|||,10|06|||,10|08|||,10|10|||,10|12|||,...,|Start L3,|Start L4,|Start L5,|Start L6,|Start R2,|Start R3,|Start R4,|Start R5,|Start R6,|Start SB
0,08|03|||,0,414,389,364,304,287,271,255,240,...,1094,1068,897,953,1080,1045,1089,913,969,994
1,08|05|||,25,0,414,389,329,312,296,280,265,...,1119,1093,922,978,1104,1070,1113,938,994,1018
2,08|07|||,50,25,0,414,354,337,321,305,290,...,1144,1118,947,1003,1129,1095,1138,963,1019,1043
3,08|09|||,75,50,25,0,379,362,346,330,315,...,1169,1143,972,1028,1154,1119,1163,988,1044,1068
4,10|04|||,484,459,434,410,0,19,35,51,67,...,1120,1074,958,838,929,1133,1087,971,854,767



Column-level summary:


,dtype,n_missing,n_unique
Unnamed: 0,object,0,1246
08|03|||,int64,0,746
08|05|||,int64,0,748
08|07|||,int64,0,743
08|09|||,int64,0,736
...,...,...,...
|Start R3,int64,0,732
|Start R4,int64,0,744
|Start R5,int64,0,688
|Start R6,int64,0,712
